In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

import nltk

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report

In [ ]:
from nltk.corpus import stopwords
from string import punctuation
from nltk.tokenize import word_tokenize

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer

from nltk.stem import WordNetLemmatizer

In [ ]:
# NN
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, Flatten, SimpleRNN

In [ ]:
df = pd.read_csv('/kaggle/input/commonlitreadabilityprize/train.csv')
df

In [ ]:
df.head()

In [ ]:
df = df[['excerpt','target']]

In [ ]:
df.isna().sum()

In [ ]:
df.info()

In [ ]:
def clean_document(doc):
    doc = doc.lower()
    # tokenization
    tokens = word_tokenize(doc)
    
    stop = stopwords.words('english')
    
    bad_tokens = stop + list(punctuation)
    
    clean_tokens = [t for t in tokens if t.lower() not in bad_tokens]
    
    
    # lemmatization
    lemma = WordNetLemmatizer()
    
    clean_tokens = [lemma.lemmatize(t) for t in clean_tokens]
    
    return ' '.join(clean_tokens)

In [ ]:
df.excerpt = df.excerpt.apply(clean_document)

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
X = df["excerpt"]
y = df["target"]

In [ ]:
X

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.1, random_state=1)

In [ ]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)

In [ ]:
vocabulary = tokenizer.index_word


In [ ]:
vocab_len = len(vocabulary)
vocab_len

In [ ]:
train_sequence = tokenizer.texts_to_sequences(X_train)

In [ ]:
doc_len = []
for doc in train_sequence:
    doc_len.append(len(doc))

In [ ]:
max(doc_len)


In [ ]:
np.quantile(doc_len, 0.99)


In [ ]:
max_len = 123


In [ ]:
# training
train_sequence_matrix = sequence.pad_sequences(train_sequence, maxlen= max_len)


In [ ]:
# testing
test_sequence = tokenizer.texts_to_sequences(X_test)
test_sequence_matrix = sequence.pad_sequences(test_sequence, maxlen= max_len)

In [ ]:
model = Sequential()
model.add(Embedding(input_dim=vocab_len+1,output_dim=100,input_length=max_len,mask_zero=True))
model.add(SimpleRNN(128,activation="relu"))
model.add(Dense(1000,activation="relu"))
model.add(Dense(800,activation="relu"))
model.add(Dense(700,activation="relu"))
model.add(Dense(520,activation="relu"))
model.add(Dense(480,activation="relu"))
model.add(Dense(320,activation="relu"))
model.add(Dense(210,activation="relu"))
model.add(Dense(128,activation="relu"))
model.add(Dense(64,activation="relu"))
model.add(Dense(1))

In [ ]:
model.compile(optimizer="adam", loss="mse")

In [ ]:
model.fit(train_sequence_matrix,y_train, batch_size=18,epochs=50)

In [ ]:
from sklearn.metrics import mean_squared_error,r2_score
y_pred = model.predict(test_sequence_matrix)
print(mean_squared_error(y_test,y_pred))
print(np.sqrt(mean_squared_error(y_test,y_pred)))
print(r2_score(y_test,y_pred))

In [ ]:
#TEst

In [ ]:
df1 = pd.read_csv('/kaggle/input/commonlitreadabilityprize/test.csv')
df1 = df1[['excerpt']]
df1.head()

In [ ]:
df1.excerpt = df1.excerpt.apply(clean_document)

In [ ]:
y = df1['excerpt']

In [ ]:
y

In [ ]:
# testing
test_sequence = tokenizer.texts_to_sequences(y)
test_sequence_matrix = sequence.pad_sequences(test_sequence, maxlen= max_len)

In [ ]:
# predict
y_pred = model.predict(test_sequence_matrix)

In [ ]:
y_pred

In [ ]:
df1 = pd.read_csv('/kaggle/input/commonlitreadabilityprize/test.csv',index_col=0)
df1['target'] = y_pred

In [ ]:
df1['target'].to_csv('submission.csv')